In [8]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,688 kB]

In [9]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-15 03:35:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.00MB/s    in 0.2s    

2020-11-15 03:35:12 (4.00 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.clzu7bsd0lkj.us-east-2.rds.amazonaws.com:5432/dataviz"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [21]:
# Read vine table from DB
total_votes_greater_than_20_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)

In [22]:
# Filter rows where total_votes >= 20
total_votes_greater_than_20_df = total_votes_greater_than_20_df.filter("total_votes >= 20")
total_votes_greater_than_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100SKP4VNPS64|          4|           16|         31|   N|                N|
|R101B0XO8UYAFV|          1|           28|         64|   N|                N|
|R101R383QPHMT0|          5|           54|         64|   N|                Y|
|R102XNXFT1X2RM|          1|            1|         28|   N|                Y|
|R106N542MSESIP|          1|          141|        146|   N|                N|
|R10BY5YZCSAQKO|          2|           63|         67|   N|                Y|
|R10CC1IHT9G7JN|          2|           14|         20|   N|                N|
|R10CXBBUL1CXQC|          2|           18|         24|   N|                Y|
|R100FQFUKK14IT|          1|           23|         26|   N|                N|
|R1019CTUOIYURH|          4|           30|         31|   N|     

In [24]:
# Create new df where helpful_votes/total_votes >= 50%
new_helpful_df = total_votes_greater_than_20_df.filter("cast(helpful_votes as float)/cast(total_votes as float) >=0.5")
new_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100SKP4VNPS64|          4|           16|         31|   N|                N|
|R101R383QPHMT0|          5|           54|         64|   N|                Y|
|R106N542MSESIP|          1|          141|        146|   N|                N|
|R10BY5YZCSAQKO|          2|           63|         67|   N|                Y|
|R10CC1IHT9G7JN|          2|           14|         20|   N|                N|
|R10CXBBUL1CXQC|          2|           18|         24|   N|                Y|
|R100FQFUKK14IT|          1|           23|         26|   N|                N|
|R1019CTUOIYURH|          4|           30|         31|   N|                N|
|R105MME4E78BTD|          1|           27|         29|   N|                Y|
| R109JJB0MU2QK|          5|           76|         80|   N|     

In [25]:
# retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = new_helpful_df.filter("vine == 'Y'")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11WA24YVPYIFT|          5|           31|         33|   Y|                N|
|R13D9ZLGOK5K46|          4|           75|         84|   Y|                N|
|R1OVB8L2G5A4EZ|          3|          110|        127|   Y|                N|
|R15KKF1D2XBERX|          4|           60|         65|   Y|                N|
|R1B694RKMRB0F6|          3|           12|         20|   Y|                N|
|R1LOB0ARENHYEN|          4|           18|         21|   Y|                N|
|R1M9Z60KAE6KE7|          4|           23|         29|   Y|                N|
|R19WIDEQ6R1WJS|          3|           16|         21|   Y|                N|
|R2374OUB2CIONO|          5|           67|         68|   Y|                N|
|R1VC63NU26XPUH|          3|           11|         21|   Y|     

In [26]:
# retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_df = new_helpful_df.filter("vine == 'N'")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100SKP4VNPS64|          4|           16|         31|   N|                N|
|R101R383QPHMT0|          5|           54|         64|   N|                Y|
|R106N542MSESIP|          1|          141|        146|   N|                N|
|R10BY5YZCSAQKO|          2|           63|         67|   N|                Y|
|R10CC1IHT9G7JN|          2|           14|         20|   N|                N|
|R10CXBBUL1CXQC|          2|           18|         24|   N|                Y|
|R100FQFUKK14IT|          1|           23|         26|   N|                N|
|R1019CTUOIYURH|          4|           30|         31|   N|                N|
|R105MME4E78BTD|          1|           27|         29|   N|                Y|
| R109JJB0MU2QK|          5|           76|         80|   N|     

In [28]:
# Determine the total number of reviews
total_reviews_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
total_number_of_reviews = total_reviews_df.count()

In [29]:
total_number_of_reviews

6106719

In [34]:
# the number of paid 5-star reviews
paid_five_star_reviews = paid_df.filter("star_rating == 5").count()
paid_five_star_reviews

1604

In [36]:
# the number of unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter("star_rating == 5").count()
unpaid_five_star_reviews

168800

In [38]:
# percentage of paid 5-star reviews
percent_paid_five_star_reviews = paid_five_star_reviews/total_number_of_reviews * 100

In [39]:
round(percent_paid_five_star_reviews, 2)

0.03

In [40]:
# percentage of unpaid 5-star reviews
percent_unpaid_five_star_reviews = unpaid_five_star_reviews/total_number_of_reviews * 100

In [41]:
round(percent_unpaid_five_star_reviews, 2)

2.76